In [ ]:
import mlflow
import mlflow.lightgbm
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import pandas as pd
import os
import random
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np
from utils import custom_train_test_split, lgbm_predict, title2filename

## MLflow 설정

In [ ]:
os.environ["LOGNAME"] = "lnh"
remote_server_uri="http://118.67.134.110:30005"
mlflow.set_tracking_uri(remote_server_uri)

client = mlflow.tracking.MlflowClient()

try:
    experiment_id = client.create_experiment(experiment_name)
except:
    experiment = client.get_experiment_by_name(experiment_name)
    experiment_id = experiment.experiment_id
experiment_id

## 데이터 불러오기 및 split

In [ ]:
SEED=13

## 1. 데이터 로딩
data_dir = '/opt/ml/input/data' # 경로
after_fe_path = os.path.join(data_dir, 'base_lgbm2.pkl')
df = pd.read_pickle(after_fe_path)

train_df = df[df.kind=='train']
train, valid = custom_train_test_split(train_df, ratio=0.7, seed=SEED) # 훈련데이터 split
test = df[df.kind=='test'] # 테스트 데이터
train2 = test[test.answerCode!=-1] # 테스트데이터 마지막 제출 2번쨰꺼까지 훈련데이터로 사용
train = pd.concat([train,train2]) # 훈련데이터 병합
train.shape, valid.shape, test.shape

x_train = train.drop('answerCode',axis=1)
y_train = train[['answerCode']]

x_valid = valid.drop('answerCode',axis=1)
y_valid = valid[['answerCode']]
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

## optuna
### 하이퍼 파라미터
- optuna.trial.Trial.suggest_categorical() : 리스트 범위 내에서 값을 선택한다.
- optuna.trial.Trial.suggest_int() : 범위 내에서 정수형 값을 선택한다.</br>
"reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 9e-2),
- optuna.trial.Trial.suggest_float() : 범위 내에서 소수형 값을 선택한다.</br>
"max_depth": trial.suggest_int("max_depth", 1, 20),
- optuna.trial.Trial.suggest_uniform() : 범위 내에서 균일분포 값을 선택한다.
- optuna.trial.Trial.suggest_discrete_uniform() : 범위 내에서 이산 균일분포 값을 선택한다.
- optuna.trial.Trial.suggest_loguniform() : 범위 내에서 로그 함수 값을 선택한다.


In [65]:
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

def objective(trial: Trial) -> float:
    # Hyper parameter 설정
    params = {
        # "max_depth": trial.suggest_int("max_depth", 1, 8),  # 8,
    #     "min_data_in_leaf": 1000,
        # "feature_fraction": 0.6,  # 0.8,
    #     "bagging_fraction": 0.75,
        # "max_cat_group": 64,
        "objective": "binary",
    #     "boosting": "gbdt",  # dart
        "learning_rate": 0.023,  # 0.01,
        # "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.05),
        # "bagging_freq": 5,
        "seed": 42,
        # "max_bin": 50,
    #     "num_leaves": 80,  # 40,
    #     "metric": "auc",
    }

    ### 피처 설정
    # 사용할 Feature 설정
    FEATS = ['uidIdx',
             'assIdx',
             'testIdx',
             'KnowledgeTag',
             'user_correct_answer',
             'user_total_answer',
             'big_category',
             'mid_category',
             'problem_num',
             'month','day','dayname','hour',
             'test_mean',
             'test_sum',
             'test_std',
             'tag_std',
             'tag_mean',
             'tag_sum',
             'solvesec_3600',
             'time_category',
             'solvesec_cumsum',
             'solvecumsum_category',
             'big_category_cumconut',
             'big_category_user_cum_acc',
             'mid_category_cumconut',
             'mid_category_user_cum_acc',
             'assess_count',
             'elo_assessmentItemID',
             'elo_problem_num',
            ]

    cat_feats = ['uidIdx','assIdx','testIdx','KnowledgeTag','big_category','mid_category',
                 'problem_num','dayname','month','time_category','solvecumsum_category']
    cat_feats_idx = [i for i,e in enumerate(FEATS) if e in cat_feats]

    ### 학습 및 예측

    date = datetime.now().strftime('%m/%d %a')
    title=f"🌈({date})[LGBM] optuna"
    using_feats=", ".join(FEATS)
    desc=f"사용된 피처({len(FEATS)})\n{using_feats}"
    cat_feats_idx, title, desc

    mlflow.lightgbm.autolog()
    lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
    lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

    model = lgb.train(
        params, 
        lgb_x_train,
        valid_sets=[lgb_x_valid],
        verbose_eval=100,
        num_boost_round=3200,
        categorical_feature=cat_feats_idx,
        early_stopping_rounds=100,
    )
    preds = model.predict(x_valid[FEATS])
    acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
    auc = roc_auc_score(y_valid, preds)
    print(f'VALID AUC : {auc} ACC : {acc}\n')
    # mlflow.log_metric("VAL AUC",auc)
    # mlflow.log_metric("VAL Set SEED",SEED)
    # model_run_id=run.info.run_id
    # post_slack("done")
    # print(f"{model_run_id=}")
    file_name = title2filename(title)
    test_preds = lgbm_predict(test, model, FEATS, f'{file_name}.csv')

    q = pd.Series(test_preds).plot(kind='kde')
    return auc

In [ ]:
from optuna.integration.mlflow import MLflowCallback
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="lgbm_parameter_opt",
    direction="maximize",
    sampler=sampler,
)

print("Optimize a model...")


remote_server_uri="http://118.67.134.110:30005"
def make_mlflow_callback():
    cb = MLflowCallback(
         tracking_uri = mlflow.get_tracking_uri(),
         metric_name="VAL AUC",
         create_experiment=True)
    return cb

mlflow_cb = make_mlflow_callback()

study.optimize(
    objective, n_trials=10, callbacks=[mlflow_cb], show_progress_bar=True
)

[I 2022-12-07 18:22:57,502] A new study created in memory with name: lgbm_parameter_opt


Optimize a model...


  0%|          | 0/10 [00:00<?, ?it/s]

2022/12/07 18:23:03 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '245a1b2819da4985afb04ad249cee05f', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current lightgbm workflow


[LightGBM] [Info] Number of positive: 1208276, number of negative: 637263
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030799 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 19967
[LightGBM] [Info] Number of data points in the train set: 1845539, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654701 -> initscore=0.639767
[LightGBM] [Info] Start training from score 0.639767
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.531253
[200]	valid_0's binary_logloss: 0.523415
[300]	valid_0's binary_logloss: 0.520911
[400]	valid_0's binary_logloss: 0.519398
[500]	valid_0's binary_logloss: 0.518468
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[600]	valid_0's binary_logloss: 0.517655
[LightGBM] [Warning] No further splits with positive

...

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 1208276, number of negative: 637263
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.031060 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 19967
[LightGBM] [Info] Number of data points in the train set: 1845539, number of used features: 30
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654701 -> initscore=0.639767
[LightGBM] [Info] Start training from score 0.639767
Training until validation scores don't improv

2022/12/07 18:31:58 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '4b73d72158c64b9a931dd9d044683bf5', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current lightgbm workflow


[LightGBM] [Info] Number of positive: 1208276, number of negative: 637263
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030970 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 19967
[LightGBM] [Info] Number of data points in the train set: 1845539, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654701 -> initscore=0.639767
[LightGBM] [Info] Start training from score 0.639767
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.557587
[200]	valid_0's binary_logloss: 0.544916
[300]	valid_0's binary_logloss: 0.540319
[400]	valid_0's binary_logloss: 0.537069
[500]	valid_0's binary_logloss: 0.534815
[600]	valid_0's binary_logloss: 0.533388
[700]	valid_0's binary_logloss: 0.532322
[800]	valid_0's binary_logloss: 0.531501
[900]	valid_0's binary_logloss: 0.530762
[100

2022/12/07 18:33:03 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '85a70111be2541d0b9a38e1ecbb01378', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current lightgbm workflow


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 1208276, number of negative: 637263
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030925 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 19967
[LightGBM] [Info] Number of data points in the train set: 1845539, number of used features: 30
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654701 -> initscore=0.639767
[LightGBM] [Info] Start training from score 0.639767
Training until validation scores don't improv